# Burner Design

This notebook is a tweakable calculator for designing a one-injector, compressed air / propane burner. It uses one-dimensional flow analysis to model system behavior.

![Burner Graphic](burnerGraphic.png)


## Inlet Parameters



In [20]:
import math
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import Markdown, display


In [77]:
#
#   Conversion Functions
#
    
# celsius to kelvin conversion
def ctok(c):
    return c+273.15
# gage psi to pascal absolute conversion
def psigtopaabs(psi):
    return (14.69595+psi)*6894.757
    

In [183]:
class Swirler:    
    #
    #   init and recalculate functions
    #
    
    def __init__(self,name,inlets,diameter,temp,pressure, k, r_spec):
        # bring in critical input values
        self.name = name
        self.inlets = inlets
        self.throatdia = diameter/1000 # convert to meters
        self.intemp = temp + 273.15 #convert to Kelvin
        self.inpressure = psigtopaabs(pressure) # convert to Pa
        self.k = k
        self.r_spec = r_spec
        
        # calculate derived values   
        self.recalc()
        
        # Initialize control widgets        
        self.wCLabel = widgets.Label(self.name)
        self.wThroat = widgets.FloatSlider(value = 0.5,
                               min = 0.1,
                               max = 1,
                               step = 0.1,
                               description = 'Dia (mm)')
        self.wIntemp = widgets.FloatLogSlider(value = 20,
                               base = 2,
                               min = .5,
                               max = 12,
                               step = 0.1,
                               description = 'Inlet (C)')
        self.wInpressure = widgets.FloatSlider(value = 5,
                               min = 0.1,
                               max = 25,
                               step = 0.1,
                               description = 'Inlet (psig)')
        self.wInlets = widgets.IntSlider(value = 1,
                               min = 1,
                               max = 16,
                               step = 1,
                               description = 'Inlet Count')
        self.wControls = widgets.VBox([self.wCLabel,self.wThroat,self.wIntemp,self.wInpressure,self.wInlets])
     
    # recalculate all derived object-scope values       
    def recalc(self):
        self.inarea = math.pi*self.throatdia**2
        self.t_crit = (2*self.intemp)/(self.k+1)
        self.v_crit = (self.k*self.r_spec*self.t_crit)**0.5
        self.p_crit = (2/(self.k+1))**(self.k/(self.k-1))*self.inpressure
        self.rho_crit = self.p_crit/(self.r_spec*self.t_crit)
        self.massflow = self.rho_crit * self.inarea * self.v_crit
        self.inarea = math.pi*(self.throatdia/2)**2 
    
    # update independent variables from controls and display outputs
    def changeIndependents(self,throatdia,temperature,pressure, inlets):
        self.throatdia = throatdia/1000
        self.intemp = ctok(temperature)
        self.inpressure = psigtopaabs(pressure)
        self.inlets = inlets
        
        self.recalc()
        self.displayParams()
    
    
    #
    #  Visualization functions
    #  
    def displayParams(self):
            
        display(Markdown("## "+self.name+" Values"))
        
        inputTable = ("Parameter | Value | Units \n"
                 "--------- | ----- | ----- \n"
                 "Pressure: | " + str(self.inpressure) +"| Pa (abs) \n"
                 "Temperature: | " + str(self.intemp) + " | Kelvin \n"
                 "Discharge Area: | " + str(self.inarea) + " | m^3")
        
        display(Markdown(inputTable))
        
        constantTable = ("Parameter | Value | Units \n"
                 "--------- | ----- | ----- \n"
                 "Spec. Gas Const: | "+ str(self.r_spec) + "| $ J \cdot kg^{-1} K^{-1} $ \n"
                 "spec. heat ratio k: | " + str(self.k) +"| dimensionless")
    
        display(Markdown(constantTable))
        
        
        outputTable = ("Parameter | Value | Units \n"
                 "--------- | ----- | ----- \n"
                 "Critical Temperature: | "+ str(self.t_crit) + "| K \n"
                 "Speed of Sound: | " + str(self.v_crit) +"| m / s \n"
                 "Critical Pressure: | " + str(self.p_crit) +"| Pa \n"
                 "Critical Density: | " + str(self.rho_crit) + " | kg / m^3 \n"
                 "Massflow: | " + str(self.massflow) + " | $ kg / s $ \n "
                 "Massflow (total): |" + str(self.massflow*self.inlets)) + "| $ kg / s $ \n "
    
        display(Markdown(outputTable))   

In [ ]:
class Combustor:
    

In [184]:
# initialize the swirlers
airNozzle = Swirler("Air",1,1,20,14,1.4,287.058)
propaneNozzle = Swirler("Propane",1,1,20,14,1.4,287.058)

# calcTogether - run calculations for both propane and air
# arguments are air throat size, air temp, air pressure, air inlet count,
# propane throat size, propane temp, propane pressure, and propane inlet count
def calcTogether(ax,at,ap,an,px,pt,pp,pn):
    airNozzle.changeIndependents(ax, ap, at, an)
    propaneNozzle.changeIndependents(ax, ap, at, an)




ui = widgets.HBox([airNozzle.wControls,propaneNozzle.wControls])

out = widgets.interactive_output(calcTogether, {'ax':airNozzle.wThroat,
                                                 'at':airNozzle.wIntemp,
                                                 'ap':airNozzle.wInpressure,
                                                 'an':airNozzle.wInlets,
                                                 'px':propaneNozzle.wThroat,
                                                 'pt':propaneNozzle.wIntemp,
                                                 'pp':propaneNozzle.wInpressure,
                                                 'pn':propaneNozzle.wInlets})

display(ui,out)





Output()